#Install requirements

In [1]:
# Install requirements
!pip install fastapi
!pip install opencv-python
# !pip install Pillow==8.3.2
!pip install timm
!pip install python-multipart
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using c

In [2]:
!pip install nest-asyncio

In [3]:
!pip install pyngrok

In [4]:
from typing import List

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse, StreamingResponse

import numpy as np

import torch

#Model

In [5]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# Đường dẫn tới thư mục chứa trọng số mô hình đã được lưu
model_path = "/content/drive/MyDrive/Machine Translation/model"

# Tải mô hình từ thư mục đã lưu
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [6]:
def generate_responses(input, from_language, to_language):
    if from_language == to_language:
        return input

    inputs = []
    if to_language == 'vi':
      inputs.append('en: ' + input);
    else:
      inputs.append('vi: ' + input);
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    # Sử dụng tokenizer để mã hóa câu đầu vào thành một tensor đầu vào cho mô hình
    input_ids = tokenizer(inputs, padding=True, return_tensors="pt")

    # Sinh ra đầu ra từ mô hình
    outputs = model.generate(input_ids.input_ids.to('cpu'), max_length=512)

    # Giải mã các đầu ra thành câu
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return decoded_outputs

#Set up server

In [7]:
# from fastapi import FastAPI, Form
# from fastapi.responses import HTMLResponse

# app = FastAPI()

# # CSS style để cải thiện giao diện
# style = """
#     <style>
#         body {
#             font-family: Arial, sans-serif;
#             margin: 0;
#             padding: 0;
#             background-color: #f4f4f4;
#             text-align: center;
#         }
#         .container {
#             max-width: 600px;
#             margin: 50px auto;
#             background-color: #fff;
#             padding: 20px;
#             border-radius: 10px;
#             box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
#         }
#         h1 {
#             color: #333;
#         }
#         textarea {
#             width: 100%;
#             height: 100px;
#             padding: 10px;
#             margin-bottom: 10px;
#             border: 1px solid #ccc;
#             border-radius: 5px;
#             resize: none;
#         }
#         input[type="submit"] {
#             padding: 10px 20px;
#             background-color: #4CAF50;
#             border: none;
#             color: white;
#             border-radius: 5px;
#             cursor: pointer;
#         }
#         input[type="submit"]:hover {
#             background-color: #45a049;
#         }
#         .translated-text {
#             margin-top: 20px;
#             text-align: left;
#         }
#         /* Cải thiện CSS cho label và select */
#         label {
#             display: block;
#             margin-bottom: 5px;
#             font-weight: bold;
#         }
#         select {
#             width: 100%;
#             padding: 10px;
#             margin-bottom: 10px;
#             border: 1px solid #ccc;
#             border-radius: 5px;
#             background-color: #fff;
#             font-size: 16px;
#             cursor: pointer;
#         }
#     </style>
# """

# @app.post("/uploadtext/")
# async def create_upload_text(text: str = Form(...), language: str = Form(...)):
#     translated_content = generate_responses(text, language)

#     # If you have the function generate_responses() properly defined
#     # translate_text = generate_responses(data)
#     # translated_content = f"<p>{translate_text[0]}</p>"

#     content = f"""
#         <body>
#             <div class="container">
#                 <h1>Translation Machine</h1>
#                 <form action="/uploadtext/" method="post">
#                     <textarea name="text" placeholder="Enter text to translate">{text}</textarea><br>
#                     <label for="language">Destination language:</label>
#                     <select name="language" id="language">
#                         <optgroup label="Choose language">
#                             <option value="en">English</option>
#                             <option value="vi">Vietnamese</option>
#                         </optgroup>
#                     </select><br>
#                     <input type="submit" value="Translate">
#                 </form>
#                 <div class="translated-text">
#                     <h3>Translated Text</h3>
#                     {translated_content}
#                 </div>
#             </div>
#         </body>
#     """

#     return HTMLResponse(content=style + content)

# @app.get("/")
# async def main():
#     content = """
#         <body>
#             <div class="container">
#                 <h1>Translation Machine</h1>
#                 <form action="/uploadtext/" method="post">
#                     <textarea name="text" placeholder="Enter text to translate"></textarea><br>
#                     <label for="language">Destination language:</label>
#                     <select name="language" id="language">
#                         <optgroup label="Choose language">
#                             <option value="en">English</option>
#                             <option value="vi">Vietnamese</option>
#                         </optgroup>
#                     </select><br>
#                     <input type="submit" value="Translate">
#                 </form>
#             </div>
#         </body>
#     """
#     return HTMLResponse(content=style + content)


In [8]:
link = """
  <body>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css">
  </body>
"""
function = """
  <script>
    function exchangeLanguages() {
        var fromLanguage = document.querySelector('select[name="from_language"]');
        var toLanguage = document.querySelector('select[name="to_language"]');
        var temp = fromLanguage.value;
        fromLanguage.value = toLanguage.value;
        toLanguage.value = temp;
    }

    function copyFromText() {
        var textArea = document.querySelector('.from-text');
        textArea.select();
        document.execCommand('copy');
    }

    function copyToText() {
        var textArea = document.querySelector('.to-text');
        textArea.select();
        document.execCommand('copy');
    }

</script>
"""

style = """
    <style>
         * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Poppins', sans-serif;
        }

        body {
            display: flex;
            align-items: center;
            justify-content: center;
            min-height: 100vh;
            background: #5372F0;
        }

        .container {
            width: 690px;
            background: #fff;
            border-radius: 7px;
            padding: 30px;
        }

        .wrapper {
            border-radius: 5px;
            border: 1px solid #bbb;
        }

        .wrapper .text-input {
            display: flex;
            border-bottom: 1px solid #bbb;
        }

        .text-input .to-text {
            border-left: 1px solid #bbb;

        }

        .text-input textarea{
            height: 250px;
            width: 100%;
            border: none;
            outline: none;
            resize: none;
            font-size: 18px;
            border-radius: 5px;
            background: none;
            padding: 10px 15px;
        }

        .controls, li, .icons, .icons i {
            display: flex;
            align-items: center;
            justify-content: space-between;
        }

        .controls {
            list-style: none;
            padding: 12px 15px;
        }

        .controls .row.from .icons{
            padding-right: 15px;
            border-right: 1px solid #bbb;
        }

        .controls .row.to .icons{
            padding-left: 15px;
            border-left: 1px solid #bbb;
        }

        .controls .icons {
            width: 38%;
        }

        .controls .icons i {
            width: 50px;
            color: #9f9f9f;
            font-size: 14px;
            cursor: pointer;
            justify-content: center;
        }

        .controls .row select {
            border: none;
            outline: none;
            font-size: 18px;
            background: none;
        }

        .controls .exchange {
            color: #9f9f9f;
            font-size: 16px;
            cursor: pointer;
        }

        .container button {
            width: 100%;
            padding: 14px;
            margin-top: 20px;
            border: none;
            outline: none;
            color: #fff;
            font-size: 17px;
            border-radius: 5px;
            background: #5372F0;
            cursor: pointer;
        }

    </style>
"""

In [9]:
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse

app = FastAPI()

@app.post("/uploadtext/")
async def create_upload_text(text: str = Form(...),
                             from_language: str = Form(...),
                             to_language: str = Form(...)):
    translated_content = generate_responses(text, from_language, to_language)
    content = f"""
    <body>
        <div class="container">
          <form action="/uploadtext/" method="post">
            <h1 style="text-align: center; margin-bottom: 20px;">Translation Machine</h1>
            <div class="wrapper">

                    <div class="text-input">
                        <textarea class="from-text" name="text" placeholder="Enter text">{text}</textarea>
                        <textarea class="to-text" placeholder="Translation" disabled>{translated_content}</textarea>
                    </div>
                    <button type="submit">Translate Text</button>

                <ul class="controls">
                    <li class="row from">
                        <div class="icons">
                            <i class="fas fa-volume-up"></i>
                            <i class="fas fa-copy" onclick="copyFromText()"></i>
                        </div>
                        <select name="from_language">
                            <option value="en">English</option>
                            <option value="vi">Vietnamese</option>
                        </select>
                    </li>
                    <li class="exchange" onclick="exchangeLanguages()"><i class="fas fa-exchange-alt"></i></li>
                    <li class="row to">
                        <select name="to_language">
                            <option value="vi">Vietnamese</option>
                            <option value="en">English</option>

                        </select>
                        <div class="icons">
                            <i class="fas fa-volume-up"></i>
                            <i class="fas fa-copy" onclick="copyToText()"></i>
                        </div>
                    </li>
                </ul>
            </div>
        </div>
      </form>
    </body>
    """
    return HTMLResponse(content=style + content + link + function)

@app.get("/")
async def main():
    content = f"""
    <body>
      <form action="/uploadtext/" method="post">
        <div class="container">
            <h1 style="text-align: center; margin-bottom: 20px;">Translation Machine</h1>
            <div class="wrapper">

                    <div class="text-input">
                        <textarea class="from-text" name="text" placeholder="Enter text"></textarea>
                        <textarea class="to-text" placeholder="Translation" disabled></textarea>
                    </div>
                    <button type="submit">Translate Text</button>

                <ul class="controls">
                    <li class="row from">
                        <div class="icons">
                            <i class="fas fa-volume-up"></i>
                            <i class="fas fa-copy"></i>
                        </div>
                        <select name="from_language">
                            <option value="en">English</option>
                            <option value="vi">Vietnamese</option>
                        </select>
                    </li>
                    <li class="exchange" onclick="exchangeLanguages()"><i class="fas fa-exchange-alt"></i></li>
                    <li class="row to">
                        <select name="to_language">
                            <option value="vi">Vietnamese</option>
                            <option value="en">English</option>
                        </select>
                        <div class="icons">
                            <i class="fas fa-volume-up"></i>
                            <i class="fas fa-copy"></i>
                        </div>
                    </li>
                </ul>
            </div>
        </div>
      </form>
    </body>
    """
    return HTMLResponse(content=style + content + link + function)

In [10]:
auth_token = "2eUB2hVXeZFJ0FaUIfa5e9nTxAD_2Q9thyJ6Dr8QUorQuyve" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
import os
os.system(f"ngrok authtoken {auth_token}")

0

In [11]:
from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000, bind_tls=True)


In [12]:
# Check if it exists
!ps aux | grep ngrok

root        1299 27.0  0.2 1256208 30880 ?       Sl   13:37   0:00 /root/.config/ngrok/ngrok start -
root        1308  0.0  0.0   7376  3492 ?        S    13:37   0:00 /bin/bash -c ps aux | grep ngrok
root        1310  0.0  0.0   6484  2408 ?        S    13:37   0:00 grep ngrok


In [ ]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

INFO:     Started server process [210]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


NgrokTunnel: "https://f8d2-34-125-132-203.ngrok-free.app" -> "http://localhost:8000"
INFO:     116.110.40.192:0 - "GET / HTTP/1.1" 200 OK
INFO:     116.110.40.192:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     116.110.40.192:0 - "POST /uploadtext/ HTTP/1.1" 200 OK
INFO:     116.110.40.192:0 - "POST /uploadtext/ HTTP/1.1" 200 OK
INFO:     116.110.40.192:0 - "POST /uploadtext/ HTTP/1.1" 200 OK


In [ ]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)